## Migrate client files

In [1]:
import inspect
import sys
sys.path.insert(0, '/src')
import json

import os
from importlib import import_module
from saiva.model.shared.utils import pascal_case
import re
path = '/src/saiva/model/clients'

In [2]:
client_files = [f[:-3] for f in os.listdir(path) if f.endswith('.py') and f != '__init__.py']

prediction_date = '2022-01-01'
facilityid = '1'
train_start_date = '2021-01-01'

In [3]:
print(client_files)

['sarrentov6', 'genhcv6', 'crn', 'carespringv6', 'venvisorv6', 'arcc', 'caldera', 'dkg', 'mhco', 'centeronaging', 'banecaremgmtv6', 'carespringv6falls', 'pghcv6alf', 'avante', 'base_matrixcare', 'base_sigmacare', 'epic_hcm', 'woodruff', 'mgmkarefirstv6', 'base', 'arccv6', 'avantev6falls', 'madp', '_template', 'banecaremgmtv6wounds']


In [4]:
prediction_date = '2022-01-01'
facilityid = '1'
train_start_date = '2021-01-01'


def get_training_queries(*, clientClass):
    try:
        queries = getattr(clientClass(), 'get_training_queries')(train_start_date, train_start_date)
    except:
        queries = getattr(clientClass(), 'get_training_queries')(
            test_end_date=train_start_date, 
            train_start_date=train_start_date,
        )
    return queries


def get_prediction_queries(*, clientClass):
    try:
        queries = getattr(clientClass(), 'get_prediction_queries')(prediction_date, facilityid, train_start_date)
    except:
        queries = getattr(clientClass(), 'get_prediction_queries')(
            prediction_date=prediction_date, 
            facilityid=facilityid, 
            train_start_date=train_start_date,
        )
    return queries


def check_unique_queries(*, data, keep_client_file):
    unique_queries = {}

    for client, queries in data.items():
        if client in keep_client_file:
            continue
        for query_name, query in queries.items():
            query = re.sub(r"\s+", "", query)

#             if query_name == 'patient_census':
#                 query = query.split("andcensusactioncodenotin(", 1)
#                 query = query[0] + query[1].split(")", 1)[1]

            unique_query = unique_queries.get(query_name, {})
            clients = unique_query.get(query, [])
            clients.append(client)
            unique_query[query] = clients
            unique_queries[query_name] = unique_query

    print(unique_queries.keys())

    for query_name, query in unique_queries.items():
        if len(query) > 1:
            print(query_name, len(query))
            for q, clients in query.items():
                print(clients)
            print('-'*100)

### Setup client names for which we want to keep the files
Update the list of client names in the cell below. The list should contain the names of the clients for which we want to keep the files.
Then run the two cells bellow the list - the otput should be empty. If not, there are some differences between the queries accross clients - please check, if you can update them to be the same, or add the client name to keep_client_file list.

In [5]:
keep_client_file = ['pghcv6alf']

In [6]:
# Check training queries

data = {}

for py in client_files:
    if py in ['base']:
        continue
    try:
        module = import_module(f'saiva.model.clients.{py}')
        clientClass = getattr(module, pascal_case(py))
        data[py] = get_training_queries(clientClass=clientClass)

    except Exception as e:
        print(py, e)

check_unique_queries(data=data, keep_client_file=keep_client_file)

base_matrixcare module 'saiva.model.clients.base_matrixcare' has no attribute 'BaseMatrixcare'
base_sigmacare module 'saiva.model.clients.base_sigmacare' has no attribute 'BaseSigmacare'
dict_keys(['master_patient_lookup', 'patient_census', 'patient_rehosps', 'patient_admissions', 'patient_demographics', 'patient_diagnosis', 'patient_vitals', 'patient_meds', 'patient_orders', 'patient_alerts', 'patient_lab_results', 'patient_progress_notes', 'patient_immunizations', 'patient_risks', 'patient_assessments', 'patient_adt'])
master_patient_lookup 2
['sarrentov6', 'genhcv6', 'carespringv6', 'venvisorv6', 'arcc', 'caldera', 'dkg', 'mhco', 'centeronaging', 'banecaremgmtv6', 'carespringv6falls', 'avante', 'mgmkarefirstv6', 'arccv6', 'avantev6falls', 'madp', '_template', 'banecaremgmtv6wounds']
['crn', 'epic_hcm', 'woodruff']
----------------------------------------------------------------------------------------------------
patient_census 2
['sarrentov6', 'genhcv6', 'carespringv6', 'venvisorv6

In [7]:
# Check prediction queries

data = {}

for py in client_files:
    if py in ['base']:
        continue
    try:
        module = import_module(f'saiva.model.clients.{py}')
        clientClass = getattr(module, pascal_case(py))
        data[py] = get_prediction_queries(clientClass=clientClass)

    except Exception as e:
        print(py, e)

check_unique_queries(data=data, keep_client_file=keep_client_file)

base_matrixcare module 'saiva.model.clients.base_matrixcare' has no attribute 'BaseMatrixcare'
base_sigmacare module 'saiva.model.clients.base_sigmacare' has no attribute 'BaseSigmacare'
_template cannot import name 'BaseClient' from 'saiva.model.clients.base' (/src/saiva/model/clients/base.py)
dict_keys(['master_patient_lookup', 'patient_vitals', 'patient_admissions', 'patient_census', 'patient_rehosps', 'patient_demographics', 'patient_diagnosis', 'patient_meds', 'patient_orders', 'patient_alerts', 'patient_lab_results', 'patient_progress_notes', 'patient_room_details', 'patient_immunizations', 'patient_risks', 'patient_assessments', 'patient_adt'])
master_patient_lookup 2
['sarrentov6', 'genhcv6', 'carespringv6', 'venvisorv6', 'arcc', 'caldera', 'dkg', 'mhco', 'centeronaging', 'banecaremgmtv6', 'carespringv6falls', 'avante', 'mgmkarefirstv6', 'arccv6', 'avantev6falls', 'madp', 'banecaremgmtv6wounds']
['crn', 'epic_hcm', 'woodruff']
---------------------------------------------------

Dump missing datasets for clients

In [7]:
data = {}

query_names = [
    'patient_census', 
    'patient_vitals', 
    'patient_admissions', 
    'master_patient_lookup', 
    'patient_rehosps', 
    'patient_demographics', 
    'patient_diagnosis', 
    'patient_meds', 
    'patient_orders', 
    'patient_alerts', 
    'patient_progress_notes', 
    'patient_lab_results', 
    'patient_room_details'
]

_queries = set()

for py in client_files:
    if py in ['base']:
        continue
    try:
        module = import_module(f'saiva.model.clients.{py}')
        clientClass = getattr(module, pascal_case(py))
        queries = get_prediction_queries(clientClass=clientClass)

        data[py] = []

        for query_name in query_names:
            query = queries.get(query_name, None)
            if not query:
                data[py].append(query_name)
                _queries.add(query_name)

    except Exception as e:
        print(py, e)

print(json.dumps(data))

print(_queries)

base_matrixcare module 'saiva.model.clients.base_matrixcare' has no attribute 'BaseMatrixcare'
base_sigmacare module 'saiva.model.clients.base_sigmacare' has no attribute 'BaseSigmacare'
{"sarrentov6": [], "genhcv6": [], "crn": ["patient_meds", "patient_orders", "patient_alerts", "patient_lab_results"], "carespringv6": [], "venvisorv6": [], "arcc": [], "caldera": [], "dkg": [], "mhco": [], "centeronaging": [], "banecaremgmtv6": [], "carespringv6falls": [], "pghcv6alf": [], "avante": [], "epic_hcm": ["patient_meds", "patient_orders", "patient_alerts", "patient_lab_results"], "woodruff": ["patient_meds", "patient_orders", "patient_alerts", "patient_lab_results"], "mgmkarefirstv6": [], "arccv6": [], "avantev6falls": [], "madp": [], "_template": [], "banecaremgmtv6wounds": []}
{'patient_alerts', 'patient_meds', 'patient_orders', 'patient_lab_results'}


Dump experiment dates

In [8]:
data = {}

for py in client_files:
    if py in ['base']:
        continue
    try:
        module = import_module(f'saiva.model.clients.{py}')
        clientClass = getattr(module, pascal_case(py))
        data[py] = getattr(clientClass(), 'get_experiment_dates')()

    except Exception as e:
        print(py, e)

print(json.dumps(data))

base_matrixcare module 'saiva.model.clients.base_matrixcare' has no attribute 'BaseMatrixcare'
base_sigmacare module 'saiva.model.clients.base_sigmacare' has no attribute 'BaseSigmacare'
{"sarrentov6": {"train_start_date": "2018-01-01", "train_end_date": "2022-11-07", "validation_start_date": "2022-11-08", "validation_end_date": "2023-04-20", "test_start_date": "2023-04-21", "test_end_date": "2023-10-09"}, "genhcv6": {"train_start_date": "2022-01-17", "train_end_date": "2023-05-04", "validation_start_date": "2023-05-05", "validation_end_date": "2023-07-23", "test_start_date": "2023-07-24", "test_end_date": "2023-10-09"}, "crn": {"train_start_date": "2021-01-01", "train_end_date": "2022-08-26", "validation_start_date": "2022-08-27", "validation_end_date": "2022-12-22", "test_start_date": "2022-12-23", "test_end_date": "2023-04-16"}, "carespringv6": {"train_start_date": "2021-05-03", "train_end_date": "2023-03-08", "validation_start_date": "2023-03-09", "validation_end_date": "2023-06-22